In [78]:
import numpy as np
from Pyfhel.util import ENCODING_t
from Pyfhel import PyCtxt, Pyfhel, PyPtxt
import requests
import time
from sklearn import datasets

In [88]:
pyfhel = Pyfhel()
pyfhel.contextGen(p=65537, m=32768, base=2, sec=128, intDigits=64, fracDigits = 32) # m = 4096
#pyfhel.contextGen(p=65537, m=2**12, base=2, sec=128, intDigits=64, fracDigits = 32)

# 2 ^ 15 = 32768
# Change m to 2 ^ 10 and 2 ^ 12 to see how it affects the runtime
pyfhel.keyGen()
pyfhel.relinKeyGen(15,3)

In [92]:
iris = datasets.load_iris()
print(len(iris.data))

150


In [93]:
test_input = np.array([6.7, 3.1, 4.7, 1.5])
result = 0.41269119*test_input[0] - 0.84800177*test_input[1] + 2.21345685*test_input[2] + 0.91860495*test_input[3] - 6.36236603

print(result)

5.555014076


In [72]:
t1 = time.time()

for a, b, c, d in iris.data:
    res = 0.41269119 * a - 0.84800177 * b + 2.21345685 * c + 0.91860495 * d - 6.36236603

d1 = time.time() - t1
print("Time elapsed for " + str(len(iris.data)) + " plaintext instances: " + "{:.6f}".format(d1) + " seconds")

ctxt = pyfhel.to_bytes_context()
pub = pyfhel.to_bytes_publicKey()
relin = pyfhel.to_bytes_relinKey()

url = 'http://35.225.238.33:8080/register_key'
files = [('context',ctxt),('public_key',pub),('relin_key',relin)]

resp = requests.post(url=url,files=files)
#print(resp.ok)

session_id = resp.json()['unique_id']

t2 = time.time()

for a, b, c, d in iris.data:
    sepal_length_ctxt = pyfhel.encryptFrac(a)
    sepal_width_ctxt = pyfhel.encryptFrac(b)
    petal_length_ctxt = pyfhel.encryptFrac(c)
    petal_width_ctxt = pyfhel.encryptFrac(d)
    
    sepal_length_bytes = sepal_length_ctxt.to_bytes()
    sepal_width_bytes = sepal_width_ctxt.to_bytes()
    petal_length_bytes = petal_length_ctxt.to_bytes()
    petal_width_bytes = petal_width_ctxt.to_bytes()
    
    url = f'http://35.225.238.33:8080/predict_encrypted/iris/{session_id}'
    files = [
        ('sepal_length',sepal_length_bytes),('sepal_width',sepal_width_bytes),
        ('petal_length',petal_length_bytes),('petal_width',petal_width_bytes)
    ]

    resp_prediction = requests.post(url=url,files=files)
    #print(resp_prediction.ok)
    
    res_bytes = resp_prediction.content
    ans_ctext = PyCtxt(serialized=res_bytes,encoding = ENCODING_t.FRACTIONAL)
    pyfhel.decryptFrac(ans_ctext)
    
d2 = time.time() - t2
print("Time elapsed for " + str(len(iris.data)) + " encrypted instances: " + "{:.6f}".format(d2) + " seconds")
print("Time difference: " + "{:.6f}".format(d2 - d1) + " seconds")

Time elapsed for 150 plaintext instances: 0.000376 seconds


ConnectionError: HTTPConnectionPool(host='35.225.238.33', port=8080): Max retries exceeded with url: /register_key (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fec6cd6dd30>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [94]:
resp_prediction

<Response [200]>

In [62]:
cnt = [len(iris.data) // 5 * i for i in range(1, 6)]

for x in cnt:
    t3 = time.time()
    for a, b, c, d in iris.data[:x]:
        sepal_length_ctxt = pyfhel.encryptFrac(a)
        sepal_width_ctxt = pyfhel.encryptFrac(b)
        petal_length_ctxt = pyfhel.encryptFrac(c)
        petal_width_ctxt = pyfhel.encryptFrac(d)

        sepal_length_bytes = sepal_length_ctxt.to_bytes()
        sepal_width_bytes = sepal_width_ctxt.to_bytes()
        petal_length_bytes = petal_length_ctxt.to_bytes()
        petal_width_bytes = petal_width_ctxt.to_bytes()

        url = f'http://35.225.238.33:8080/predict_encrypted/iris/{session_id}'
        files = [
            ('sepal_length',sepal_length_bytes),('sepal_width',sepal_width_bytes),
            ('petal_length',petal_length_bytes),('petal_width',petal_width_bytes)
        ]

        resp_prediction = requests.post(url=url,files=files)
        #print(resp_prediction.ok)

        res_bytes = resp_prediction.content
        ans_ctext = PyCtxt(serialized=res_bytes,encoding = ENCODING_t.FRACTIONAL)
        pyfhel.decryptFrac(ans_ctext)
    
    d3 = time.time() - t3
    print("Time elapsed for " + str(x) + " encrypted instances: " + "{:.6f}".format(d3) + " seconds")

Time elapsed for 30 encrypted instances: 1.326004 seconds
Time elapsed for 60 encrypted instances: 2.630698 seconds
Time elapsed for 90 encrypted instances: 3.951652 seconds
Time elapsed for 120 encrypted instances: 5.216906 seconds
Time elapsed for 150 encrypted instances: 6.536748 seconds


In [95]:
weights = [0.41269119,-0.84800177,2.21345685,0.91860495]
bias = - 6.36236603

p_weights = [pyfhel.encode(weight) for weight in weights]
p_bias = pyfhel.encode(bias)

In [97]:
t1 = time.time()

for a, b, c, d in iris.data:
    res = 0.41269119 * a - 0.84800177 * b + 2.21345685 * c + 0.91860495 * d - 6.36236603

d1 = time.time() - t1
print("Time elapsed for " + str(len(iris.data)) + " plaintext instances: " + "{:.6f}".format(d1) + " seconds")

a, b, c, d = iris.data[0]
sepal_length_ctxt = pyfhel.encryptFrac(a)
sepal_width_ctxt = pyfhel.encryptFrac(b)
petal_length_ctxt = pyfhel.encryptFrac(c)
petal_width_ctxt = pyfhel.encryptFrac(d)

t2 = time.time()

for a, b, c, d in iris.data:
        ans = pyfhel.encryptFrac(0.0)
        encrypted_params = [sepal_length_ctxt,sepal_width_ctxt,petal_length_ctxt,petal_width_ctxt]
        for p_txt_w,input_c_txt in zip(p_weights,encrypted_params):
            ans += pyfhel.multiply_plain(input_c_txt,p_txt_w,in_new_ctxt=True)
        ans_val = pyfhel.decryptFrac(ans)
        
d2 = time.time() - t2
print("Time elapsed for " + str(len(iris.data)) + " ciphertext instances: " + "{:.2f}".format(d2) + " seconds")
print("Time difference: " + "{:.2f}".format(d2 - d1) + " seconds")

Time elapsed for 150 plaintext instances: 0.000702 seconds


KeyboardInterrupt: 

In [91]:
ans = pyfhel.encryptFrac(0.0)
encrypted_params = [sepal_length_ctxt,sepal_width_ctxt,petal_length_ctxt,petal_width_ctxt]
for p_txt_w,input_c_txt in zip(p_weights,encrypted_params):
    ans += pyfhel.multiply_plain(input_c_txt,p_txt_w,in_new_ctxt=True)
ans_val = pyfhel.decryptFrac(ans)

ValueError: encrypted is not valid for encryption parameters

In [98]:
cnt = [len(iris.data) // 5 * i for i in range(1, 6)]

a, b, c, d = iris.data[0]
sepal_length_ctxt = pyfhel.encryptFrac(a)
sepal_width_ctxt = pyfhel.encryptFrac(b)
petal_length_ctxt = pyfhel.encryptFrac(c)
petal_width_ctxt = pyfhel.encryptFrac(d)

for x in cnt:
    t3 = time.time()
    for a, b, c, d in iris.data[:x]:
        ans = pyfhel.encryptFrac(0.0)
        encrypted_params = [sepal_length_ctxt,sepal_width_ctxt,petal_length_ctxt,petal_width_ctxt]
        for p_txt_w,input_c_txt in zip(p_weights,encrypted_params):
            ans += pyfhel.multiply_plain(input_c_txt,p_txt_w,in_new_ctxt=True)
        ans_val = pyfhel.decryptFrac(ans)
    
    d3 = time.time() - t3
    print("Time elapsed for " + str(x) + " encrypted instances: " + "{:.2f}".format(d3) + " seconds")

Time elapsed for 30 encrypted instances: 11.50 seconds
Time elapsed for 60 encrypted instances: 22.94 seconds
Time elapsed for 90 encrypted instances: 34.47 seconds
Time elapsed for 120 encrypted instances: 45.95 seconds
Time elapsed for 150 encrypted instances: 57.46 seconds


In [ ]:
sepal_length_ctxt,sepal_width_ctxt,petal_length_ctxt,petal_width_ctxt = [pyfhel.encryptFrac(inp) for inp in test_input]

In [ ]:
sepal_length_bytes = sepal_length_ctxt.to_bytes()
sepal_width_bytes = sepal_width_ctxt.to_bytes()
petal_length_bytes = petal_length_ctxt.to_bytes()
petal_width_bytes = petal_width_ctxt.to_bytes()

In [ ]:
ctxt = pyfhel.to_bytes_context()
pub = pyfhel.to_bytes_publicKey()

In [ ]:
url = 'http://35.225.238.33:8080/register_key'
files = [('context',ctxt),('public_key',pub)]

resp = requests.post(url=url,files=files)
print(resp.ok)

In [ ]:
session_id = resp.json()['unique_id']

In [ ]:
url = f'http://35.225.238.33:8080/predict_encrypted/iris/{session_id}'
files = [
    ('sepal_length',sepal_length_bytes),('sepal_width',sepal_width_bytes),
    ('petal_length',petal_length_bytes),('petal_width',petal_width_bytes)
]

resp_prediction = requests.post(url=url,files=files)
print(resp_prediction.ok)

In [ ]:
res_bytes = resp_prediction.content

In [ ]:
ans_ctext = PyCtxt(serialized=res_bytes,encoding = ENCODING_t.FRACTIONAL)

In [ ]:
pyfhel.decryptFrac(ans_ctext)